In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn
from tqdm import tqdm

In [2]:
df = pd.read_csv('train.csv')
featureimportance=pd.read_csv("featureimportance")
featureimportance= pd.Series(featureimportance['target'].values, index=featureimportance['Unnamed: 0'])

In [3]:
# featuretarget= df.drop(['row_id', 'time_id', "investment_id"], axis=1)
# corr_matrix=featuretarget.corr()
# a=corr_matrix["target"].sort_values()
# b=abs(a)
# c=b.sort_values()

In [4]:
featurename1=list(featureimportance.index[250:300])
featurename2=list(featureimportance.index[200:250])
featurename3=list(featureimportance.index[150:200])
featurename4=list(featureimportance.index[100:150])

In [5]:
feature1 = df[featurename1]
feature2 = df[featurename2]
feature3 = df[featurename3]
feature4 = df[featurename4]

y = df["target"]
id=df["investment_id"]

In [6]:
del df

In [7]:
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature1=torch.from_numpy(feature1.values)
feature2=torch.from_numpy(feature2.values)
feature3=torch.from_numpy(feature3.values)
id=torch.from_numpy(id.values)
y=torch.from_numpy(y.values)


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to(device)
        self.y = y.squeeze().to(device)
        self.id = id.squeeze().to(torch.int64).to(device)
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_1=FinData(feature=feature1,id=id,y=y)
Dataset_2=FinData(feature=feature2,id=id,y=y)
Dataset_3=FinData(feature=feature3,id=id,y=y)

DataLoader1=DataLoader(Dataset_1,batch_size=512,shuffle=True)
DataLoader2=DataLoader(Dataset_2,batch_size=512,shuffle=True)
DataLoader3=DataLoader(Dataset_3,batch_size=512,shuffle=True)

In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,512)
        self.linear1=nn.Linear(512,32)

        self.linear3=nn.Linear(50,128)
        self.linear4=nn.Linear(128,64)

        self.linear5=nn.Linear(96,64)
        self.linear8=nn.Linear(64,1)

        self.linear2=nn.Linear(96,2)


        self.act2= nn.Softmax(dim=1)

        

        self.dropout=nn.Dropout(p=0.5)

    def forward(self, feature, id):
        #id feature construction
        id=self.embedding(id).squeeze()
        id=self.dropout(id)
        id=self.linear1(id).squeeze()
        id=id*torch.sigmoid(id)
    
        

        #other feature construction
        feature=self.linear3(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.dropout(feature)
        
        #combine them all together and do some the regresson and clasification at the same time
        x=torch.cat((id,feature),1)

        #the regression
        x1=self.linear5(x).squeeze()
        x1=x1*torch.sigmoid(x1)
        x1=self.linear8(x1).squeeze()
        x1=20*torch.sigmoid(x1)
        #the classification 
        x2=self.linear2(x).squeeze()
        
        
  

        return x1,x2


In [11]:
model1_1 = NeuralNetwork().to(device)
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to(device)
model1_2=model1_2.double()

model2_1 = NeuralNetwork().to(device)
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to(device)
model2_2=model2_2.double()


model3_1 = NeuralNetwork().to(device)
model3_1=model3_1.double()
model3_2 = NeuralNetwork().to(device)
model3_2=model3_2.double()

In [12]:
learning_rate = 1e-3
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_2 = torch.optim.Adam(model2_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer3_1 = torch.optim.Adam(model3_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer3_2 = torch.optim.Adam(model3_2.parameters(), lr=learning_rate,weight_decay=1e-5)
loss_fn1=torch.nn.MSELoss()
loss_fn2=torch.nn.CrossEntropyLoss()

In [13]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        x1,label = model(feature,id)
        x1=x1.to("cuda")
        label=label.to("cuda")
        loss1 = loss_fn1(x1,torch.abs(y))
        loss2 = loss_fn2(label,(0.5*(y/torch.abs(y)+torch.tensor([1]).to(device))).long().to("cuda"))
        loss = loss1+0.1*loss2
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def train_2(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        x1,label = model(feature,id)
        loss1 =  -torch.corrcoef(torch.stack((x1,torch.abs(y))))[0,1]
        loss2 = loss_fn2(label,(0.5*(y/torch.abs(y)+torch.tensor([1]).to(device))).long().to("cuda"))
        loss = loss1+0.1*loss2
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    


In [14]:
for i in range(10):
    print("epoch{}/{}".format(i+1,10))
    train_1(DataLoader1,model1_1,optimizer1_1)
    train_2(DataLoader1,model1_2,optimizer1_2)
    train_1(DataLoader2,model2_1,optimizer2_1)
    train_2(DataLoader2,model2_2,optimizer2_2)
    train_1(DataLoader3,model3_1,optimizer3_1)
    train_2(DataLoader3,model3_2,optimizer3_2)

epoch1/10


100%|██████████| 6136/6136 [00:46<00:00, 133.23it/s]


epoch2/10


100%|██████████| 6136/6136 [00:47<00:00, 127.84it/s]


epoch3/10


...

100%|██████████| 6136/6136 [00:46<00:00, 133.02it/s]


epoch9/10


100%|██████████| 6136/6136 [00:45<00:00, 133.40it/s]


epoch10/10


100%|██████████| 6136/6136 [00:45<00:00, 133.76it/s]


In [15]:
torch.save(model1_1, 'less feature with auxiliary task/model1_1.pth')
torch.save(model1_2, 'less feature with auxiliary task/model1_2.pth')
torch.save(model2_1, 'less feature with auxiliary task/model2_1.pth')
torch.save(model2_2, 'less feature with auxiliary task/model2_2.pth')
torch.save(model3_1, 'less feature with auxiliary task/model3_1.pth')
torch.save(model3_2, 'less feature with auxiliary task/model3_2.pth')

In [16]:
test_id=id[1:50].squeeze().to(torch.int64).to(device)

In [17]:
test_feature1=feature1[1:50].to(device)

In [18]:
label,x=model1_1(test_feature1,test_id)

In [19]:
x[1]

tensor([ 0.1483, -0.0883], device='cuda:0', dtype=torch.float64,
       grad_fn=<SelectBackward0>)

In [20]:
x


tensor([[ 0.0653, -0.2020],
        [ 0.1483, -0.0883],
        [ 0.2556,  0.1136],
        [ 0.4594,  0.4036],
        [ 0.3263,  0.0048],
        [ 0.0709, -0.0631],
        [ 0.4260,  0.0909],
        [ 0.4202, -0.2323],
        [ 0.2835, -0.1279],
        [ 0.1904, -0.2591],
        [ 0.1449, -0.0753],
        [ 0.2340, -0.0035],
        [ 0.3652,  0.0377],
        [ 0.3992, -0.2087],
        [ 0.2015, -0.1804],
        [ 0.4457,  0.1997],
        [ 0.1584, -0.0161],
        [ 0.0974, -0.0197],
        [ 0.3165, -0.0692],
        [ 0.0305, -0.2525],
        [ 0.1666, -0.0592],
        [ 0.1064, -0.1851],
        [ 0.2714, -0.1641],
        [ 0.2939, -0.0435],
        [ 0.1304, -0.1302],
        [ 0.2858, -0.3914],
        [ 0.1269, -0.1060],
        [ 0.3687, -0.0714],
        [ 0.1841, -0.2819],
        [ 0.3313, -0.0110],
        [ 0.2358, -0.0170],
        [ 0.0933,  0.0933],
        [ 0.2634,  0.0015],
        [ 0.1212, -0.1151],
        [ 0.1931, -0.1431],
        [ 0.1326,  0

In [21]:
for a in x:
    if a[0]>a[1]:
        print(1)
    else:
        print(-1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [22]:
a=torch.tensor([1,2,3])
b=torch.tensor([2,2,3])

In [23]:
(a+b)/2

tensor([1.5000, 2.0000, 3.0000])

In [24]:
1/4*a

tensor([0.2500, 0.5000, 0.7500])

In [25]:
a.dim()

1

In [26]:
torch.cat(([1,2,3],[1,2,3]),dim=0)

TypeError: expected Tensor as element 0 in argument 0, but got list

tensor([[ 0.6311,  0.1867, -0.1014,  ...,  0.5837,  0.2014,  3.2514],
        [-0.4045,  0.7268, -0.2553,  ...,  0.5419, -0.4918, -0.2698],
        [-0.0792, -0.1675, -0.2702,  ...,  0.5863, -0.0924, -0.2698],
        ...,
        [-0.7135,  1.6104,  0.9973,  ..., -1.2391,  0.2432, -0.2413],
        [-0.3049,  0.3086, -0.3551,  ...,  0.8754, -0.7416, -0.2412],
        [ 0.0335,  0.9868, -0.2740,  ...,  0.7431, -0.2005, -0.2370]],
       dtype=torch.float64)